### Лабораторна робота No2
### Наука про дані: підготовчий етап

In [175]:
import pandas as pd
import urllib.request
import os
from datetime import datetime

### Завантаження файлів

In [176]:
# Функція для перевірки, чи файл вже існує
def file_exists(province_id, save_dir):
    # Шукаємо файли, які містять індекс області у назві
    for filename in os.listdir(save_dir):
        if f"vhi_province_{province_id}" in filename:
            return os.path.join(save_dir, filename)
    return None

# Функція для завантаження файлів 
def download_data(id, save_dir="vhi_data"):

    for i in range(1, id + 1):
        url = f'https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php?country=UKR&provinceID={i}&year1=1981&year2=2024&type=Mean'
        
        # Формування імені файлу з датою і часом
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f'vhi_province_{i}_{timestamp}.csv'
        filepath = os.path.join(save_dir, filename)

        # Перевіряємо, чи файл для даної області вже існує
        existing_file = file_exists(i, save_dir)
        if existing_file:
            continue  # Пропускаємо завантаження, якщо файл існує

        # Завантажуємо файл і зберігаємо його
        vhi_url = urllib.request.urlopen(url)
        with open(filepath, 'wb') as out:
            out.write(vhi_url.read())
            
    print("Data is downloaded")

download_data(27, save_dir="vhi_data")


Data is downloaded


### Об'єднання файлів в один DataFrame

In [177]:
# Функція для видалення конкретних HTML-тегів
def clean_html_tags(text):
    clean_text = text.replace('<tt>', '').replace('</tt>', '').replace('<pre>', '').replace('</pre>', '')
    return clean_text

def load_files_to_dataframe(directory_path):
    # Створюємо порожній список для збереження DataFrame з кожного файлу
    df_list = []
    i = 0
    
    # Проходимо по всіх файлах у директорії
    for filename in os.listdir(directory_path):
        if filename.endswith(".csv"):  # Обробляємо тільки файли CSV
            file_path = os.path.join(directory_path, filename)
            i += 1
            headers = ['Year', 'Week', 'SMN', 'SMT', 'VCI', 'TCI', 'VHI', 'empty']
            df = pd.read_csv(file_path, header=1, names=headers)
            
            # Очищуємо всі стовпці від HTML-тегів
            for col in df.columns:
                if df[col].dtype == 'object':
                    df[col] = df[col].map(lambda x: clean_html_tags(str(x)))
        
            # Додаємо стовбчик з індексом регіона
            df['area'] = i
            
            # Додаємо DataFrame до списку
            df_list.append(df)
    
    # Об'єднуємо всі DataFrame в один
    combined_df = pd.concat(df_list, ignore_index=True)
    
    # Змінюємо тип даних в Year
    combined_df['Year'] = pd.to_numeric(dataframe['Year'], errors='coerce')
    combined_df.dropna(subset=['Year'], inplace=True)
    combined_df['Year'] = dataframe['Year'].astype(int)
    
    return combined_df

dataframe = load_files_to_dataframe('vhi_data')
dataframe.head()

,Year,Week,SMN,SMT,VCI,TCI,VHI,empty,area
0,1982,1.0,0.059,258.24,51.11,48.78,49.95,NaN,1
1,1982,2.0,0.063,261.53,55.89,38.20,47.04,NaN,1
2,1982,3.0,0.063,263.45,57.30,32.69,44.99,NaN,1
3,1982,4.0,0.061,265.10,53.96,28.62,41.29,NaN,1
4,1982,5.0,0.058,266.42,46.87,28.57,37.72,NaN,1


### Змінення індексів регіонів

In [178]:
def change_index(df):
    # Створимо словник відповідностей
    replacement_dict = {1: 22, 2: 24, 3: 23, 4: 25, 5: 3, 6: 4, 7: 8, 8: 19, 9: 20, 10: 21, 11: 9, 12: 12, 13: 10, 14: 11, 15: 12, 16: 13, 17: 14, 18: 15, 19: 16, 20: 20, 21: 17, 22: 18, 23: 6, 24: 1, 25: 2, 26: 7, 27: 5}
    
    # Використовуємо заміну 
    df["area"] = df["area"].replace(replacement_dict)
    
    return df


dataframe = change_index(dataframe)
dataframe.head()

,Year,Week,SMN,SMT,VCI,TCI,VHI,empty,area
0,1982,1.0,0.059,258.24,51.11,48.78,49.95,NaN,22
1,1982,2.0,0.063,261.53,55.89,38.20,47.04,NaN,22
2,1982,3.0,0.063,263.45,57.30,32.69,44.99,NaN,22
3,1982,4.0,0.061,265.10,53.96,28.62,41.29,NaN,22
4,1982,5.0,0.058,266.42,46.87,28.57,37.72,NaN,22


### Ряд VHI для області за вказаний рік

In [179]:
def get_vhi_for_area(dataframe, area, year):
    result = dataframe[(dataframe['area'] == area) & (dataframe['Year'] == year)]['VHI']
    return result


get_vhi_for_area(dataframe, area=22, year=1982)

0     49.95
1     47.04
2     44.99
3     41.29
4     37.72
5     34.91
6     33.14
7     32.72
8     32.77
9     32.23
10    30.38
11    31.12
12    31.65
13    32.61
14    35.49
15    39.19
16    41.14
17    39.50
18    37.07
19    37.88
20    40.99
21    43.36
22    45.31
23    46.30
24    48.85
25    50.88
26    51.83
27    51.68
28    51.61
29    49.93
30    46.00
31    43.56
32    41.20
33    38.42
34    39.22
35    39.13
36    37.25
37    36.38
38    35.99
39    34.87
40    29.96
41    28.16
42    27.39
43    25.05
44    23.80
45    22.82
46    24.41
47    27.34
48    28.53
49    27.87
50    29.83
51    31.99
Name: VHI, dtype: float64

### Пошук екстремумів (min та max) для вказаних областей та років, середнього, медіани

In [180]:
def get_extremes_mean_median(dataframe, areas, years):
    # Перевіряємо чи areas та years є списками
    if not isinstance(areas, list):
        areas = [areas]
    if not isinstance(years, list):
        years = [years]
    
    # Фільтрація даних для вказаних областей та років
    filtered_df = dataframe[dataframe['area'].isin(areas) & dataframe['Year'].isin(years)]
    
    if filtered_df.empty:
        print("Немає даних для зазначених областей та років.")
        return None

    # Обчислення екстремумів, середнього значення та медіани
    min_vhi = filtered_df['VHI'].min()
    max_vhi = filtered_df['VHI'].max()
    mean_vhi = filtered_df['VHI'].mean()
    median_vhi = filtered_df['VHI'].median()

    results = {
        'min': min_vhi,
        'max': max_vhi,
        'mean': mean_vhi,
        'median': median_vhi,
    }

    return results


extremes = get_extremes_mean_median(dataframe, [22,18], [1982])
if extremes:
    print("Results:")
    print(f"min: {extremes['min']}")
    print(f"max: {extremes['max']}")
    print(f"average: {extremes['mean']}")
    print(f"median: {extremes['median']}")

Results:
min: 22.82
max: 64.88
average: 41.03798076923077
median: 39.055


### Ряд VHI за вказаний діапазон років для вказаних областей

In [181]:
def get_vhi_for_area_range(dataframe, areas, start_year, end_year):
    # Переконайтеся, що areas є списком
    if not isinstance(areas, list):
        areas = [areas]
    
    # Фільтрація даних для вказаних областей та діапазону років
    filtered_df = dataframe[(dataframe['area'].isin(areas)) & (dataframe['Year'].between(start_year, end_year))]
    
    if filtered_df.empty:
        print("Немає даних для зазначених областей та діапазону років")
        return None

    return filtered_df[['Year', 'area', 'VHI']]


get_vhi_for_area_range(dataframe, 22, 1982, 1986)

,Year,area,VHI
0,1982,22,49.95
1,1982,22,47.04
2,1982,22,44.99
3,1982,22,41.29
4,1982,22,37.72
...,...,...,...
255,1986,22,26.04
256,1986,22,31.28
257,1986,22,35.85
258,1986,22,39.25


### Роки, назви областей з екстремальними посухами та значення VHI

In [185]:
def detect_droughts(dataframe, perсent):
    # Фільтруємо дані, щоб отримати записи з VHI <= 15
    df_drought = dataframe[(dataframe.VHI <= 15) & (dataframe.VHI != -1)]

    # Обчислюємо кількість областей, які підпадають під екстремальні посухи за роками
    drought_counts = df_drought.groupby('Year')['area'].nunique()

    # Знаходимо роки, де кількість областей з посухами перевищує вказаний відсоток 
    significant_drought_years = drought_counts[drought_counts > (27 * perсent / 100)]

    # Отримаємо дані для повернення
    results = []

    for year in significant_drought_years.index:
        drought_data = df_drought[df_drought['Year'] == year]
        for index, row in drought_data.iterrows():
            results.append({'Year': year, 'Area': row['area'], 'VHI': row['VHI']})

    # Створюємо DataFrame з результатами
    return pd.DataFrame(results)


detect_droughts(dataframe, 20)

,Year,Area,VHI
0,2000,24.0,12.51
1,2000,24.0,10.60
2,2000,24.0,11.20
3,2000,24.0,12.32
4,2000,24.0,14.65
5,2000,23.0,14.89
6,2000,23.0,12.76
7,2000,23.0,7.81
8,2000,23.0,6.49
9,2000,23.0,6.58
